In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("../Dataset/Raw_Data.csv", encoding='latin-1')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
df.shape

(541909, 8)

In [5]:
df.describe

<bound method NDFrame.describe of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

            InvoiceDate  UnitPrice  Custo

In [7]:
#Handling missing data 
df=df.dropna(subset=["CustomerID"])


In [8]:
df.shape

(406829, 8)

In [9]:
#Filling missing values
df["UnitPrice"]=df["UnitPrice"].fillna(0)
df["Quantity"]=df["Quantity"].fillna(0)

In [10]:
#Removing orders when quantity is 0
df=df[df["Quantity"]>0]

In [11]:
#Removing invoice data that starts with C
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

In [12]:
# Changing Date to datetime format 
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"])

In [13]:
#Create new feature total purchase 
df["TotalPurchase"]=df['Quantity'] * df['UnitPrice']
Cleaned_df=df

In [14]:
df= df.sort_values('InvoiceDate')
cutoff_date= pd.Timestamp('2011-06-30')
prediction_end= pd.Timestamp('2011-12-31')

In [18]:
past_data=df[df['InvoiceDate']<= cutoff_date]
future_data=df[(df['InvoiceDate'] > cutoff_date) & 
                (df['InvoiceDate']<= prediction_end)]

## Creating Customer df

In [19]:
#Creating snapshot to use recency
snapshot_date = past_data['InvoiceDate'].max() + pd.Timedelta(days=1)

In [27]:
# Group by customer
customer_df = past_data.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,# Recency
    'InvoiceNo': 'nunique',# Frequency
    'TotalPurchase': 'sum'# Monetary Value
}).reset_index()

In [28]:
customer_df.head()

,CustomerID,InvoiceDate,InvoiceNo,TotalPurchase
0,12346.0,163,1,77183.60
1,12347.0,21,4,2205.95
2,12348.0,86,3,1487.24
3,12350.0,148,1,334.40
4,12352.0,100,5,1561.81


In [29]:
# Rename columns
customer_df.rename(columns={
    'InvoiceDate': 'Recency',
    'InvoiceNo': 'Frequency',
    'TotalPurchase': 'MonetaryValue'
}, inplace=True)

In [30]:
customer_df.head()

,CustomerID,Recency,Frequency,MonetaryValue
0,12346.0,163,1,77183.60
1,12347.0,21,4,2205.95
2,12348.0,86,3,1487.24
3,12350.0,148,1,334.40
4,12352.0,100,5,1561.81


In [31]:
rfm = customer_df

In [32]:
future_revenue= (future_data.groupby('CustomerID')['TotalPurchase'].sum().reset_index())

In [33]:
future_revenue.columns=['CustomerID', 'Future_6M_Revenue']

In [35]:
model_df=rfm.merge(future_revenue,on='CustomerID',how='left')
model_df['Future_6M_Revenue']= model_df['Future_6M_Revenue'].fillna(0)

In [36]:
rfm.to_csv("rfm.csv", index=False)